In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch.nn.functional as F
import pandas as pd
import torch
import math

In [4]:
target_sentences_df = pd.read_csv('/content/drive/MyDrive/PhD/target_sentences.csv')
target_sentences_df

,Text
0,još malo pa će izbori
1,naoblačilo se kao da će padati kiša
2,naša kola su stara 10 godina
3,otišao je u inostranstvo prošle godine i još s...
4,telefoni zvone po ceo dan i tebe traže
...,...
62,pojavio se u poslednjem trenutku
63,sanjao sam te pre neku noć
64,premijer je obećao viši standard
65,sutra je doček Nove Godine


In [5]:
vocabulary_df = pd.read_csv('/content/drive/MyDrive/PhD/wordlist_classlawiki_sr_cleaned.csv')
vocabulary_df

,word,freq
0,biti,5835707
1,u,4289454
2,i,2848044
3,na,1602609
4,sebe,1544095
...,...,...
336153,"1,77.",5
336154,"0,038",5
336155,"0,036",5
336156,"0,032",5


In [6]:
pip install transformers

In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

model_name = 'gpt2'

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

**Similarity Functions**

In [8]:
def non_context_embedding(word, vord, model, tokenizer, j):

  # Tokenize and get embedding for the target word
  word_input_ids = tokenizer.encode(word, return_tensors='pt')

  with torch.no_grad():
    outputs = model(word_input_ids, output_hidden_states=True)
    # Access hidden_states and get the last layer
    word_embedding = outputs.hidden_states[-j].mean(dim=1)

  vocab_input_ids = tokenizer.encode(vord, return_tensors='pt')
  with torch.no_grad():
    outputs = model(vocab_input_ids, output_hidden_states=True)
    # Access hidden_states and get the last layer
    vocab_embedding = outputs.hidden_states[-j].mean(dim=1)

  # Compute cosine similarity and normalize
  similarity = 0.5 * (F.cosine_similarity(word_embedding, vocab_embedding).item() + 1)

  return similarity

**Information Value**

In [9]:
def extract_words_and_embeddings(subwords, subword_embeddings):

    words = []
    word_embedding = []

    current_word = ""
    current_embedding = []

    for subword, embedding in zip(subwords, subword_embeddings):
        # Check if the subword starts with the special token 'Ġ'
        if subword.startswith('Ġ'):
            if current_word:
                words.append(current_word)
                word_embedding.append(sum(current_embedding) / len(current_embedding))

            # Reset current word and probability for the new word
            current_word = subword[1:]  # Remove 'Ġ' from the start
            current_embedding= [embedding]
        else:
            # Concatenate subwords to form the current word
            current_word += subword
            # Multiply probabilities for subwords within the same word
            current_embedding.append(embedding)

    # Add the last word and its probability
    if current_word:
        words.append(current_word)
        word_embedding.append(sum(current_embedding) / len(current_embedding))

    return words, word_embedding

In [10]:
def extract_words_and_probabilities(subwords, subword_probabilities):

    words = []
    word_probabilities = []

    current_word = ""
    current_probability = 1.0  # Initialize to 1.0 as we will multiply probabilities

    # Iterate through the subword probabilities
    for subword, probability in zip(subwords, subword_probabilities):
        # Check if the subword starts with the special token 'Ġ'
        if subword.startswith('Ġ'):
            # If we have a current word, add it to the list with its probability
            if current_word:
                words.append(current_word)
                word_probabilities.append(current_probability)

            # Reset current word and probability for the new word
            current_word = subword[1:]  # Remove 'Ġ' from the start
            current_probability = probability
        else:
            # Concatenate subwords to form the current word
            current_word += subword
            # Multiply probabilities for subwords within the same word
            current_probability *= probability

    # Add the last word and its probability
    if current_word:
        words.append(current_word)
        word_probabilities.append(current_probability)

    return words, word_probabilities

In [11]:
def calculate_word_adjusted_surprisal(sentence, vocabulary_df, model = model, tokenizer = tokenizer):

    # calulate information value for one sentence
    words_list = sentence.split(' ')
    print(sentence)

    ce_iv = [[],[],[],[],[],[],[],[],[],[],[],[],[]]
    nce_iv = [[],[],[],[],[],[],[],[],[],[],[],[],[]]

    words = sentence.split(' ')
    input_ids = tokenizer.encode(" ".join(words), return_tensors='pt')

    # Forward pass to get hidden states
    with torch.no_grad():
      outputs = model(input_ids, output_hidden_states=True)

    decoded_subwords = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

    for j in range(1,13):

      last_hidden_state = outputs.hidden_states[-j]
      decoded_words1, embeddings1 = extract_words_and_embeddings(decoded_subwords, last_hidden_state[0,:])
      print(f"Layer:{j}")

      # loop through all words in sentence
      for i in range(0, len(sentence.split(' '))):

        words = sentence.split(' ')
        word = words[i]
        print(word)

        ce_iv[j].append(0)
        nce_iv[j].append(0)

        embedding_word1 = embeddings1[i]

        vocab_df = vocabulary_df.sample(n=50, random_state=42).reset_index(drop=True)

        # loop through all vocabulary words
        for vord in vocab_df['word'].tolist():

          words[i] = vord
          # Tokenize the input sentence
          input_ids = tokenizer.encode(" ".join(words), return_tensors='pt')

          # Generate word probabilities using GPT-2 model
          with torch.no_grad():
              outputs = model(input_ids, output_hidden_states=True)
              logits = outputs.logits
              last_hidden_state = outputs.hidden_states[-j]  # This is the final layer's hidden state for each token

          # Extract probabilities for each word
          subword_probabilities = torch.softmax(logits, dim=-1).mean(dim=2)

          # Decode the tokens back to words
          decoded_subwords = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
          decoded_words2, probabilities = extract_words_and_probabilities(decoded_subwords, subword_probabilities[0])

          decoded_words2, embeddings2 = extract_words_and_embeddings(decoded_subwords, last_hidden_state[0,:])
          embedding_word2 = embeddings2[i]

          context_probability = probabilities[decoded_words2==vord].item()

          ce_iv[j][i] += 0.5 * (F.cosine_similarity(embedding_word1, embedding_word2, dim=0).item() + 1) * context_probability
          nce_iv[j][i] += non_context_embedding(word, vord, model, tokenizer, j) * context_probability

        ce_iv[j][i] = - math.log(ce_iv[j][i] + 1e-40, 2)
        nce_iv[j][i] = - math.log(nce_iv[j][i] + 1e-40, 2)

    return words_list, ce_iv, nce_iv

In [ ]:
words_list = []
target_sentence_list = []
ce_iv_list = [[],[],[],[],[],[],[],[],[],[],[],[],[]]
nce_iv_list = [[],[],[],[],[],[],[],[],[],[],[],[],[]]

# Save the DataFrame to a CSV file
csv_file_path = "/content/drive/MyDrive/PhD/embedding_adjusted_surprisal1.csv"

for i in range(35,50):
  sentence = target_sentences_df['Text'][i].lower()
  print(i)
  words, ce_ivs, nce_ivs = calculate_word_adjusted_surprisal(sentence.strip(), vocabulary_df)

  for ind in range(0,len(words)):
    words_list.append(words[ind])
    target_sentence_list.append(i)
    for j in range(1,13):
      ce_iv_list[j].append(ce_ivs[j][ind])
      nce_iv_list[j].append(nce_ivs[j][ind])

  # Create a DataFrame
  df = pd.DataFrame({
      'Sentence': target_sentence_list,
      'Word': words_list,
      **{f'CE {j}': ce_iv_list[j] for j in range(1, 13)},
      **{f'NCE {j}': nce_iv_list[j] for j in range(1, 13)}
                     })
  df.to_csv(csv_file_path, index=False)

# Display the DataFrame
print(df)

35
nismo u kontaktu više od godinu dana
Layer:1
nismo
u
kontaktu
više
od
godinu
dana
Layer:2
nismo
u
kontaktu
više
od
godinu
dana
Layer:3
nismo
u
kontaktu
više
od
godinu
dana
Layer:4
nismo
u
kontaktu
više
od
godinu
dana
Layer:5
nismo
u
kontaktu
više
od
godinu
dana
Layer:6
nismo
u
kontaktu
više
od
godinu
dana
Layer:7
nismo
u
kontaktu
više
od
godinu
dana
Layer:8
nismo
u
kontaktu
više
od
godinu
dana
Layer:9
nismo
u
kontaktu
više
od
godinu
dana
Layer:10
nismo
u
kontaktu
više
od
godinu
dana
Layer:11
nismo
u
kontaktu
više
od
godinu
dana
Layer:12
nismo
u
kontaktu
više
od
godinu
dana
36
u braku smo već devet godina
Layer:1
u
braku
smo
već
devet
godina
Layer:2
u
braku
smo
već
devet
godina
Layer:3
u
braku
smo
već
devet
godina
Layer:4
u
braku
smo
već
devet
godina
Layer:5
u
braku
smo
već
devet
godina
Layer:6
u
braku
smo
već
devet
godina
Layer:7
u
braku
smo
već
devet
godina
Layer:8
u
braku
smo
već
devet
godina
Layer:9
u
braku
smo
već
devet
godina
Layer:10
u
braku
smo
već
devet
godina
Layer:11
u
bra